In [30]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

# Read in data
z_data = pd.read_csv('fse17_act_deact_zscore_groups.csv',sep=';',decimal='.')

# Remove irrelevant columns
z_data.drop(list(z_data.filter(regex = '_deact')),axis = 1, inplace = True)


#first, drop everything non-numeric ===========================================================
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('response', axis = 1)
z_data = z_data.drop('snippet', axis = 1)

numLabels = 2 # num of different labels (comprehension and rest)
sizeTrainSet = 11 # num of participants used for training
sizeTestSet = 3 # num participants used for testing
numParticipants = sizeTrainSet + sizeTestSet

In [2]:
#group the data and get the labels after the grouping
#return the grouped data and the labels
def prepare(data,groupingColumns):
    grouped = data.groupby(groupingColumns)
    groupedAgg = grouped.aggregate(np.mean)
    labels = groupedAgg.index.get_level_values(level='task')
    return groupedAgg,labels

In [3]:
#split according to participants (first 12 and last 4)
#return all four sets (much like train_test_split from sklearn)
#TODO: introduce variation (see file coarseAverageParticipantSplit)

def split(features, labels, low, high):
    
    #=== creating training and validation set ======================================================
    # Saving feature names for later use
    feature_list = list(features.columns)

    # Convert to numpy array
    features = np.array(features)
    labels = np.array(labels)

    #get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
    training_features = features[0:low]
    training_features = np.array(training_features)

    testing_features = features[low:high]
    testing_features = np.array(testing_features)

    training_target = labels[0:low]
    training_target = np.array(training_target)
    testing_target = labels[low:high]
    testing_target = np.array(testing_target)
    #=== end creating training and validation set ===================================================
    
    return training_features, testing_features, training_target, testing_target

In [4]:
# assumes that the column names of the structure xxx_number_xxx, e.g., aggr_13_groups0
def findIndices(name):
    numGroups = int(name.split('_')[1])
    low = sizeTrainSet*numGroups*numLabels
    high = numParticipants*numGroups*numLabels
    return low, high

In [34]:
colName = 'aggr_13_groups0'
columnsForGrouping = ['proband', 'task',colName]
features, labels = prepare(z_data,columnsForGrouping)
offset = z_data.columns.get_loc(colName)
low, high = findIndices(colName)
training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
#=============================================================================================================
print(offset)
print(low, high)
print(testing_features.shape)
print(features.shape)
print(labels.shape)
print(columnsForGrouping)
# features

422
286 364
(78, 390)
(364, 390)
(364,)
['proband', 'task', 'aggr_13_groups0']


68-177-121_act  68-180-121_act  68-183-121_act  \
proband task aggr_13_groups0                                                   
ab87    0    eightth               -0.091628       -0.281788       -0.182527   
             eleventh               0.539451        0.328748       -0.143597   
             fifth                  0.280249        0.074099       -0.014945   
             first                 -0.137228       -0.284458       -0.277643   
             fourth                 0.065869       -0.178433       -0.278951   
             nineth                 0.277469        0.095169        0.173131   
             second                -0.019306       -0.186899       -0.336380   
             seventh                0.308541        0.038164       -0.214396   
             sixth                  0.329704        0.496373        0.275014   
             tenth                  0.164839       -0.057519       -0.084370   
             third                 -0.250037       -0.262233       -0.095451   
             thirteenth             0.341676        0.086077       -0.298236   
             twelveth               0.775338        0.348231       -0.132125   
        1    eightth               -0.361905        0.038164        0.535065   
             eleventh              -0.223628       -0.104422       -0.052011   
             fifth                  0.167977        0.013702       -0.230281   
             first                 -0.481037       -0.387790       -0.258522   
             fourth                 0.152255        0.283047        0.283578   
             nineth                -0.061184        0.198717        0.548205   
             second                -0.089925        0.220654        0.203922   
             seventh               -0.254969        0.007857        0.294528   
             sixth                  0.083415        0.437996        0.450735   
             tenth                 -0.935638       -0.780695       -0.188508   
             third                 -0.160184        0.217623        0.595371   
             thirteenth            -0.473504       -0.333526       -0.367857   
             twelveth              -0.055669       -0.079166       -0.206061   
bo49    0    eightth                0.062692        0.294617        0.399743   
             eleventh               0.083759        0.170628       -0.513050   
             fifth                  0.049632        0.202403        0.267642   
             first                 -0.053890        0.218615        0.742426   
...                                      ...             ...             ...   
pe80_b  1    tenth                  0.484624        0.701096        0.742781   
             third                 -0.439605       -0.513938       -0.498850   
             thirteenth             0.045845       -0.285671       -0.287246   
             twelveth               0.362699        0.750072        0.846433   
pg74    0    eightth               -0.446881       -0.775029       -0.999417   
             eleventh               0.270614        0.094632        0.235339   
             fifth                  0.274144       -0.006548        0.167937   
             first                 -0.298473       -0.547303       -0.575751   
             fourth                -0.162084       -0.429275       -0.478706   
             nineth                -0.327959       -0.576896       -0.573401   
             second                 0.394315        0.177680        0.181736   
             seventh                0.072085       -0.217928       -0.241323   
             sixth                  0.319866       -0.110453        0.191592   
             tenth                 -0.593986       -0.490560       -0.880915   
             third                  0.193939        0.048786       -0.093100   
             thirteenth            -0.260299       -0.275892       -0.379310   
             twelveth               0.574164        0.212440        0.198567   
        1    eightth          

In [8]:
import pdb

#define the columns to group by
groupingColumn1 = 'proband'
groupingColumn2 = 'task'
groupingColumn3 = ''

#define how many different variants of aggregation and aggregation levels
numVariationsPerAggLevel = 10
numAggLevels = 4
numAggregationVariants = numVariationsPerAggLevel * numAggLevels

#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')
print(offset)
collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
#     pdb.set_trace()
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================
    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    print(offset)
    print(low, high)
    
    folder = 'Act/'
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'
    #=============================================================================================================
    # run t_pot
    # TODO: extract method
    tpot = TPOTClassifier(generations=5, population_size=5, verbosity=2, n_jobs=20,max_eval_time_mins=1)
    tpot.fit(training_features, training_target)
    print(tpot.score(testing_features, testing_target))
    tpot.export(folder + fileName)
#     #=============================================================================================================
    
    offset = offset + 1


392
392
44 56


C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: 0.95


Generation 2 - Current best internal CV score: 0.975


Generation 3 - Current best internal CV score: 0.975


...

Generation 2 - Current best internal CV score: 0.9066502463054187


Generation 3 - Current best internal CV score: 0.9066502463054187


Generation 4 - Current best internal CV score: 0.9066502463054187


Generation 5 - Current best internal CV score: 0.9066502463054187



Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=20, p=2, weights=uniform)
0.9743589743589743


In [9]:
#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')
print(offset)
collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):    

    #=============================================================================================================
    columnsForGrouping = ['proband','task',z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================
    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'

    #=============================================================================================================
    #do the learning
    # TODO: extract method
    with open(folder + fileName) as f:
        content = f.readlines()
    cleanedContent = []
    for line in content:
        if 'tpot_data' not in line and 'training_target, testing_target' not in line:
            cleanedContent.append(line)
    
    fileNameCleaned = folder + 'cleaned_' + fileName
    with open(fileNameCleaned, 'w') as filehandle:  
        for line in cleanedContent:
            filehandle.write('%s\n' % line)
            
    %run -i $fileNameCleaned
    
    # print the accuracy
    # TODO: extract method
    # TODO: print all accurracy values to one file
    num_matches = 0;
    for a, b in zip(testing_target, results):
        if a == b:
            num_matches = num_matches + 1
        else:
            unmatched.append((a,b))
    print('Number of matches:',num_matches,'(of',testing_target.size,')')

    accuracy = num_matches/testing_target.size*100
    print('Accuary: ',accuracy)
    #=============================================================================================================
    
    #=============================================================================================================
    #collect all accuracy values to plot them
    collectAccuracies.append(accuracy)
    
    
    offset = offset + 1
    

392
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 11 (of 12 )
Accuary:  91.66666666666666
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 9 (of 12 )
Accuary:  75.0
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 8 (of 12 )
Accuary:  66.66666666666666
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 12 (of 12 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 17 (of 18 )
Accuary:  94.44444444444444
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 18 (of 18 )
Accuary:  100.0
Number of matches: 51 (

In [10]:
len(unmatched)

92

In [11]:
countComprehensionRest = 0
countComprehensionSyntax = 0
countRestSyntax = 0
for x in unmatched:
    if x == (0,1) or x == (1,0):
        countComprehensionRest = countComprehensionRest + 1
    if x == (0,2) or x == (2,0):
        countComprehensionSyntax = countComprehensionSyntax + 1
    if x == (1,2) or x == (2,1):
        countRestSyntax = countRestSyntax + 1
print('Comprehension/Rest',countComprehensionRest)
print('Comprehension/Syntax',countComprehensionSyntax)
print('Rest/Syntax',countRestSyntax)

Comprehension/Rest 92
Comprehension/Syntax 0
Rest/Syntax 0


In [12]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [13]:
list1, list2, list3, list4 = split_list(collectAccuracies,4)
frames = [list1, list2, list3, list4]
collectedAccuracies = pd.DataFrame(frames).T
collectedAccuracies.columns = ['2_groups_act','3_groups_act','9_groups_act','13_groups_act']


collectedAccuracies.to_csv('triangulate_Act.csv',sep=';',decimal='.', index=False)